In [19]:
'''
This is to build a model which aligns the image of the camera with steering 
input to centre the car in the lane

It requires images with relevant steering inputs already generated
'''
import keras
import os
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Conv2D, concatenate, Embedding, Reshape, Flatten, Activation
from keras.optimizers import SGD
import numpy as np

#constants to resize image to
HEIGHT = 360
WIDTH = 640

#get a lsit of files
mypath = 'C:/SelfDrive/_out'
only_images = [f.split('.')[0] for f in os.listdir(mypath) if f.endswith(".png")]
only_steering = [f.split('.')[0] for f in os.listdir(mypath) if f.endswith(".str")]
# get a list when both are available: image and steering
complete_names = set(only_images).intersection(only_steering)

# read training data 

X = [] #images
Y = [] #expected steering for this image
for example in complete_names:
    img_path = mypath+'/'+example+'.png'
    str_path = mypath+'/'+example+'.str'
     
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    # option to make images smaller
    image = cv2.resize(image, (WIDTH,HEIGHT))
    
    X.append(image[:, :, None] / 255) # adding another dimension and normalising pixels to 0-1
    # max value of Y during writing of this was 0.79 so no need to convert it to 0-1
    
    f = open(str_path,'r')
    y = f.readline()
    Y.append(float(y))

X = np.array(X)
Y = np.array(Y)

In [21]:
model = Sequential()
model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(HEIGHT, WIDTH,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['MSE'])


model.fit(X, Y, batch_size=32, epochs=3, validation_split=0.3)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 360, 640, 8)       80        
                                                                 
 activation_6 (Activation)   (None, 360, 640, 8)       0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 180, 320, 8)      0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 460800)            0         
                                                                 
 dense_4 (Dense)             (None, 32)                14745632  
                                                                 
 activation_7 (Activation)   (None, 32)                0         
                                                      

In [ ]:
# need to get to 0.002 and below to be within 5% ish

In [1]:
'''
next iteration:

deal with activation at the end to allow negative values - by re-scaling Y labels to center around 0.5

focus on certain screen area

'''
import keras
import os
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Conv2D, concatenate, Embedding, Reshape, Flatten, Activation
from keras.optimizers import SGD
import numpy as np

#constants to resize image to
HEIGHT = 360
WIDTH = 640

HEIGHT_REQUIRED_PORTION = 0.4 #bottom share, e.g. 0.1 is take lowest 10% of rows
WIDTH_REQUIRED_PORTION = 0.5

height_from = int(HEIGHT * (1 -HEIGHT_REQUIRED_PORTION))
width_from = int((WIDTH - WIDTH * WIDTH_REQUIRED_PORTION) / 2)
width_to = width_from + int(WIDTH_REQUIRED_PORTION * WIDTH)

#get a lsit of files
mypath = 'C:/SelfDrive/_out'
only_images = [f.split('.')[0] for f in os.listdir(mypath) if f.endswith(".png")]
only_steering = [f.split('.')[0] for f in os.listdir(mypath) if f.endswith(".str")]
# get a list when both are available: image and steering
complete_names = set(only_images).intersection(only_steering)

# read training data 

X = [] #images
Y = [] #expected steering for this image
for example in complete_names:
    img_path = mypath+'/'+example+'.png'
    str_path = mypath+'/'+example+'.str'
     
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    # option to make images smaller
    image = cv2.resize(image, (WIDTH,HEIGHT))
    # this version adds taking lower side of the image
    image = image[height_from:,width_from:width_to]
    
    X.append(image[:, :, None] / 255) # adding another dimension and normalising pixels to 0-1
    # max value of Y during writing of this was 0.79 so no need to convert it to 0-1
    
    f = open(str_path,'r')
    y = f.readline()
    Y.append(float(y))

X = np.array(X)
Y = np.array(Y)
Y = Y/2 + 0.5 # transforming Y from [-1:+1] to [0:1]

In [3]:
#initial model giving good results - needs to be checked against a real driving scenario
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())  
model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['MSE'])


model.fit(X, Y, batch_size=32, epochs=5, validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 144, 320, 64)      640       
                                                                 
 activation_5 (Activation)   (None, 144, 320, 64)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 72, 160, 64)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 160, 64)       36928     
                                                                 
 activation_6 (Activation)   (None, 72, 160, 64)       0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 80, 64)       0         
 2D)                                                  

In [2]:
#adding an extra layer - no difference - not saved
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())  
model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['MSE'])


model.fit(X, Y, batch_size=32, epochs=3, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 144, 320, 64)      640       
                                                                 
 activation (Activation)     (None, 144, 320, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 72, 160, 64)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 160, 64)       36928     
                                                                 
 activation_1 (Activation)   (None, 72, 160, 64)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 80, 64)       0         
 2D)                                                    

In [8]:
cv2.imshow("image", image)
cv2.waitKey(0)

-1

In [4]:
#saving model
model.save("lane_model_360x640_04_05", overwrite=False,include_optimizer=True,
    save_format=None, signatures=None, options=None, save_traces=True)

INFO:tensorflow:Assets written to: lane_model_360x640_04_05\assets


INFO:tensorflow:Assets written to: lane_model_360x640_04_05\assets
